In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_auto_biweekly 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH asm_kg_tv AS (
    SELECT DISTINCT st.business_unit, a.store_id, a.product_id
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_scd a 
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store st USING(store_id)
    WHERE (
            (st.business_unit = '2000' AND a.assortment IN ('VMP_KGMB', 'VMP_KGMN', 'WMP_THOIVU'))
            OR (st.business_unit = '1500' AND (a.assortment LIKE 'KG\\_%' OR a.assortment LIKE 'VM\\_KG%'))
            OR a.assortment LIKE 'KG_%'
        )
        AND a.status = 'ACTIVE'
        AND DATE(start_date) <= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) 
        AND DATE(end_date) >= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
), 
tac AS (
    SELECT * FROM {catalog_name}.udp_wcm_silver_dim.wmt_assortment_clustering 
    WHERE proc_date = (SELECT MAX(proc_date) FROM {catalog_name}.udp_wcm_silver_dim.wmt_assortment_clustering)
), 
pac AS (
    SELECT * FROM {catalog_name}.udp_wcm_silver_dim.wmp_assortment_clustering 
    WHERE proc_date = (SELECT MAX(proc_date) FROM {catalog_name}.udp_wcm_silver_dim.wmp_assortment_clustering)
), 
tab02a AS (
    SELECT *
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_active_biweekly
    WHERE calday = (SELECT MAX(calday) FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_active_biweekly)
)
SELECT 
    st.business_unit, a.store_id, a.product_id,
    t1.core_range assortment_type,
    CASE
        WHEN t1.core_range IN ('Core 1', 'Core 2') OR t2.product_id IS NOT NULL THEN IFNULL(tac.mdq, IFNULL(pac.mdq, 2))
        ELSE NULL
    END mdq
FROM tab02a a 
LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store st ON st.store_id = a.store_id
LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product pd ON pd.product_id = a.product_id
LEFT JOIN tac ON tac.bu = st.business_unit AND tac.mch5_id = pd.mch5_id AND tac.size_st = st.size_st
LEFT JOIN pac ON pac.bu = st.business_unit AND pac.city = st.city AND pac.product_id = pd.product_id
LEFT JOIN asm_kg_tv akt ON akt.business_unit = st.business_unit AND akt.store_id = a.store_id AND akt.product_id = a.product_id
LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_core_range_assortment t1 ON a.store_id = t1.store_id AND a.product_id = t1.product_id
LEFT JOIN (SELECT DISTINCT store_id, product_id FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_hero) t2 ON a.store_id = t2.store_id AND a.product_id = t2.product_id
WHERE 
    (
        pd.mch2_id <> '101' 
        OR a.product_id IN (SELECT DISTINCT product_id FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_winare_dc_fresh_sku)
    ) 
    AND akt.product_id IS NULL
    AND st.business_unit IN ('1500', '2000', '6000');
""")

In [0]:
spark.sql(f"""
DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_auto_biweekly_hist 
WHERE CALDAY = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)""")


In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_auto_biweekly_hist
SELECT 
    DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) AS calday,
    business_unit,
    store_id,
    product_id,
    mdq
FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_auto_biweekly;
""")